In [4]:
# pip install azure-search-documents --pre --upgrade
# pip install azure-search --pre --upgrade
# pip install azure-core --pre --upgrade

In [5]:
# Import libs

import os  
import json  
# from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  

In [6]:
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

In [7]:
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  

In [8]:
# Configure environment variables  

service_endpoint = "https://...search.windows.net"
index_name = "..."
key = "..." 
credential = AzureKeyCredential(key)

In [9]:
# Create embeddings

from sentence_transformers import SentenceTransformer  
  
model = SentenceTransformer('intfloat/e5-small-v2')  
  
with open('./data/text-sample.json', 'r', encoding='utf-8') as file:  
    input_data = json.load(file)  
  
for item in input_data:  
    title = item['title']  
    content = item['content']  
    title_embeddings = model.encode(title, normalize_embeddings=True)  
    content_embeddings = model.encode(content, normalize_embeddings=True)  
    item['titleVector'] = title_embeddings.tolist()  
    item['contentVector'] = content_embeddings.tolist()  
  
with open("./output/docVectors-e5.json", "w") as f:  
    json.dump(input_data, f)  

In [10]:
# Create a search index  
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)  
fields = [  
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),  
    SearchableField(name="title", type=SearchFieldDataType.String),  
    SearchableField(name="content", type=SearchFieldDataType.String),  
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),  
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
                searchable=True, vector_search_dimensions=384, vector_search_profile_name="myHnswProfile"),  
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
                searchable=True, vector_search_dimensions=384, vector_search_profile_name="myHnswProfile"),  
]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(  
            name="myHnsw",  
            kind=VectorSearchAlgorithmKind.HNSW,  
            parameters=HnswParameters(  
                m=4,  
                ef_construction=400,  
                ef_search=500,  
                metric=VectorSearchAlgorithmMetric.COSINE,  
            )  
        ),  
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",   
        ),  
    ],  
)  
  
semantic_config = SemanticConfiguration(  
    name="my-semantic-config",  
    prioritized_fields=SemanticPrioritizedFields(  
        title_field=SemanticField(field_name="title"),  
        keywords_fields=[SemanticField(field_name="category")],  
        content_fields=[SemanticField(field_name="content")]  
    )  
)  
  
# Create the semantic settings with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])  
  
# Create the search index with the semantic settings  
index = SearchIndex(name=index_name, fields=fields,  
                    vector_search=vector_search, semantic_search=semantic_search)  
result = index_client.create_or_update_index(index)  
print(f'{result.name} created')  

ai102srch231123 created


In [12]:
# Insert text and embeddings into vector store

with open('./output/docVectors-e5.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 108 documents


In [13]:
# Perform a vector similarity search

query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,
    vector_queries= [vector_query], 
    select=["title", "content", "category"],  
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n") 

Title: Azure DevOps
Score: 0.84249896
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure Mobile Apps
Score: 0.8417595
Content: Azure Mobile Apps is a mobile app development platform that enables you to build, test, deploy, and monitor your mobile applications. It provides features like offline data sync, push notifications, and user authentication. Mobile Apps supports various platforms, including iOS,

In [14]:
# Perform a cross-field vector search

query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector, titleVector")
  
results = search_client.search(  
    search_text=None,
    vector_queries= [vector_query], 
    select=["title", "content", "category"],  
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.03306011110544205
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.03279569745063782
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports vari

In [15]:
# Perform a multi-vector search

query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query_1 = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="titleVector")
vector_query_2 = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],  
    select=["title", "content", "category"],  
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.03306011110544205
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.03279569745063782
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports vari

In [16]:
# Perform a pure vector search with a filter

query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,
    vector_queries= [vector_query], 
    filter="category eq 'Developer Tools'",
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    select=["title", "content", "category"],  
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps
Score: 0.84249896
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8416291
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such

In [20]:
# Perform a semantic hybrid search

query = "tools for software development"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
query_embeddings = model.encode(query, normalize_embeddings=True) 
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Title: Azure DevOps
Reranker Score: 3.2264091968536377
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools
Caption: Developer Tools. Azure DevOps is a suite of services that help you plan, build, and deploy applications.

Title: Azure DevTest Labs
Reranker Score: 2.981311321258545
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in 